In [5]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen
import torch
import random
import numpy as np
from tqdm import tqdm_notebook as tqdm
from collections import defaultdict
import seaborn as sns
from urllib.request import urlopen
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import pickle
from sentence_transformers import SentenceTransformer
from time import gmtime, strftime
import concurrent.futures

In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english", truncation=True,max_seq_length=512)
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
# Load pre-trained models
model_name = 'bert-base-nli-mean-tokens'
model_enc = SentenceTransformer(model_name)

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english", truncation=True,max_seq_length=512)
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

def filter_(data):
    return data.replace("\n","").replace("\t","").replace(",","").replace(".","")

def negative_or_positive(sentence):
    inputs = tokenizer(sentence, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_class_id = logits.argmax().item()
    return model.config.id2label[predicted_class_id] == 'POSITIVE'

def make_embedings(sents):
    return model_enc.encode(sents)

#####################

bad_list = []
good_list = []
asin_list = []
same_asin_list = []

f = []
with gzip.open('Books.json.gz') as file:
    for k, i in enumerate(file):
        seq = json.loads(i.strip())
        try:
            if 40 < len(seq['reviewText']) < 256:
                f.append(i)
                if k == 100000:
                    break
        except KeyError:
            ...

for k, l in enumerate(tqdm(f, total = 51311621)):
    same_asin_list = []
    good_list = []
    bad_list = []
    a = json.loads(l.strip())
    if a['asin'] in asin_list:
        continue
        # print('continue on ', a['asin'])
    else:
        asin_list.append(a['asin'])
        for kk, ll in enumerate(f):
            b = json.loads(ll.strip())
            if b['asin'] == a['asin'] and b['reviewText'] != a['reviewText']:
                same_asin_list.append(b)
                # print('append', b['asin'])
        #лист отзывов об одном и том же товаре собран, приступаем к разделению по оценке
        # и проверка тональности с помощью классификатора
        for i in same_asin_list:
            if int(i['overall']) <= 2:
                if negative_or_positive(i['reviewText'][:512]) == 0:
                    bad_list.append(filter_(i['reviewText']))
                    # print(bad_list)
            if int(i['overall']) >= 4:
                if negative_or_positive(i['reviewText'][:512]) == 1:
                    good_list.append(filter_(i['reviewText'])
                                     )
        if len(bad_list) != 0 and len(good_list) != 0:
            bad_list_vec = make_embedings(bad_list)
            good_list_vec = make_embedings(good_list)
            #Расчет косинусного расстояния и составление пар
            batch_size = 2
            batch_num = len(bad_list_vec) // batch_size
            sims_part = np.array
            for k in range( len(bad_list_vec) // batch_size ):
                start = k * batch_size
                stop =  (k + 1) * batch_size

C:\Users\bulat\AppData\Local\Temp\ipykernel_9436\2961441801.py:43: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for k, l in enumerate(tqdm(f, total = 51311621)):


  0%|          | 0/51311621 [00:00<?, ?it/s]

KeyboardInterrupt: 